#### Checklist
##### Cleaning
- [] check unique values for each columns  
- [] remove oulier (percentile)  
- [] remove unused columns  
- [] check empty, none, nan values, white spaces (regex)  

##### Additional datas
- [] add province and zip code (external csv datas)  


#### Questions

- percentage of houses by provinces within a budget  
- price per sq meters by price  
- graph heatmap over belgium (price/sqmeter)  


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
# import geopandas as gpd


In [ ]:
file = './datas/Immoweb_data_ok_maite.csv'
data = pd.read_csv(file)
data.head(5)

1. Check for duplicated rows in the df

In [ ]:
data.duplicated()
data.isnull().sum()

### Cleaning data



1. Replace 'none' values with pd.NA
2. Remove rows where the price is missing

In [ ]:
data.replace(np.NaN,'None', inplace=True)
data.replace('None', pd.NA, inplace=True)
data.head()
# Remove rows where Price is not present
data = data.drop(data[data['Price'].isna()].index)
data.shape[0]


3. Remove data from apartment_group and house_group

In [ ]:
data[data['House or appartment?'] == 'APARTMENT_GROUP'].count()
data.drop(data[data['House or appartment?'] == 'APARTMENT_GROUP'].index, inplace=True)
data[data['House or appartment?'] == 'HOUSE_GROUP'].count()
data.drop(data[data['House or appartment?'] == 'HOUSE_GROUP'].index, inplace=True)

data['House or appartment?'].unique()


4. Check subtype: remove all unnecessary subtypes.  
Only keep Houses and Apartment subtype for the analysis (abandonned)

In [ ]:
# # Define array with the value of Subtype to keep.
# array = ["HOUSE", "APARTMENT"]
# # Drop all rows where subtype is not in the array
# data.drop(data.loc[~data['Subtype'].isin(array)].index, inplace=True)

data['Subtype'].unique()



### Add new columns
    -price sq meter

In [ ]:
# Remove rows where Living area is missing
data.drop((data[data['Living area'].isna()].index), inplace=True)
# Check if no more rows with missing 'Living area'
data['Living area'].isna().sum()

# Creat new column with price per square meter
data['price_square_meters'] = (data['Price'] / data['Living area'])
# Make sure the data type of the new column is numeric
data['price_square_meters'] = (pd.to_numeric(data['price_square_meters'], errors='coerce')).round(2)

data['price_square_meters'].info()


### Correlation : practice

In [ ]:
sns.set_theme(style="white")
corr = data.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

### Remove outliers from Price and price_square_meters

In [ ]:

q1_price , q3_price = data['Price'].quantile([0.25,0.75])
iqr_price = q3_price - q1_price
q1_pricesqm , q3_pricesqm = data['price_square_meters'].quantile([0.25,0.75])
iqr_pricesqm = q3_pricesqm - q1_pricesqm

iqr_coef = 1
data = data[(data['Price'] > (q1_price - iqr_coef*iqr_price)) & (data['Price'] < ((q3_price + iqr_coef*iqr_price)))]
data = data[(data['price_square_meters'] > (q1_pricesqm - iqr_coef*iqr_pricesqm)) & (data['price_square_meters'] < ((q3_pricesqm + iqr_coef*iqr_pricesqm)))]

# g_price = sns.regplot(x='Price', y='price_square_meters',  data=data)

plt.show()


### Lm Plot Price per sqm versus Price, using SeaBorn

In [ ]:
data['price_square_meters'] = data['price_square_meters'].astype(float)
data['Price'] = data['Price'].astype(float)
sns.set_context('talk')

# Graph
g_price_linearreg = sns.lmplot(x='Price', y='price_square_meters',hue='Subtype',hue_order=['HOUSE','APARTMENT'], data=data, line_kws={'alpha':1},scatter_kws={'alpha':0.1}, height=12)

# Tile and labels
g_price_linearreg.fig.suptitle('Price per square meter related to the price,\n over Belgium', y=1.05, size=24 )
g_price_linearreg.set_ylabels('Price per square meter, €/m²')
g_price_linearreg.set_xlabels('Price, €')

plt.show()

